<a href="https://colab.research.google.com/github/ballban/cryptopals/blob/main/cryptopals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Function

In [ ]:
%pip install PyCryptoDome

In [ ]:
import base64
from collections import defaultdict
import string
import requests
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import random
import os
import re
import urllib.parse

In [ ]:
def GetFrequencyDic(text):
  fre_bytes = defaultdict(int)
  for i in range(len(text)):
    x = text[i]
    fre_bytes[x] += 1
  return sorted(fre_bytes.items(), key=lambda x:x[1], reverse=True)

In [ ]:
def GetScore(text):
  score_dic = {"E":11.16,"A":8.50,"R":7.58,"I":7.54,"O":7.16,"T":6.95,"N":6.65,"S":5.74,"L":5.49,"C":4.54,"U":3.63,"D":3.38,"P":3.17,"M":3.01,"H":3.00,"G":2.47,"B":2.07,"F":1.81,"Y":1.78,"W":1.29,"K":1.10,"V":1.01,"X":0.29,"Z":0.27,"J":0.20,"Q":0.20,"e":11.16,"a":8.50,"r":7.58,"i":7.54,"o":7.16,"t":6.95,"n":6.65,"s":5.74,"l":5.49,"c":4.54,"u":3.63,"d":3.38,"p":3.17,"m":3.01,"h":3.00,"g":2.47,"b":2.07,"f":1.81,"y":1.78,"w":1.29,"k":1.10,"v":1.01,"x":0.29,"z":0.27,"j":0.20,"q":0.20," ":18}
  score = 0
  for key, value in GetFrequencyDic(text):
    if key in score_dic:
      score += score_dic[key] * value
    elif key not in string.printable:
      score = 0
      break
  return score

In [ ]:
def TryDecrypt(text):
  if(type(text) == str):
    text_bytes = bytes.fromhex(text)
  else:
    text_bytes = text
  #print(text_bytes)

  keys = range(0, 127)
  scores = defaultdict(int)
  for key in keys:
    decrypt_text = ''
    for i in range(len(text_bytes)):
      x = text_bytes[i]
      decrypt_text += chr(x ^ key)
    scores[key] = GetScore(decrypt_text)
  scores = sorted(scores.items(), key=lambda x:x[1], reverse=True)
  #print(scores[:10])

  true_key = scores[0][0]

  result = ''
  for i in range(len(text_bytes)):
      x = text_bytes[i]
      result += chr(x ^ true_key)
  return scores[0][1], result, true_key

In [ ]:
def GetTxt(url):
  f = requests.get(url).content.decode('utf-8')
  text_list = f.split('\n')
  return text_list

In [ ]:
def Encrypt(text, key):
  key = key * int(len(text) / len(key) + 1)
  result = b''
  for i in range(len(text)):
    result += (ord(text[i]) ^ ord(key[i])).to_bytes(1, 'big')
  return result

In [ ]:
def HammingDistance(text1, text2):
  # text1 = bytes(text1, 'utf-8')
  # text2 = bytes(text2, 'utf-8')
  return sum([bin(x[0] ^ x[1]).count("1") for x in zip(text1, text2)])

In [127]:
def decrypt_ECB(text: bytes, key) -> bytes:
  cipher = AES.new(key, AES.MODE_ECB)
  return cipher.decrypt(text)

def encrypt_ECB(text: bytes, key, block_size: int) -> bytes:
  cipher = AES.new(key, AES.MODE_ECB)
  encrypted_bytes = b''
  for i in range(len(text)//block_size + 1):
    chunk = text[i*block_size:(i+1)*block_size]
    if(len(chunk) == 0):
      return encrypted_bytes
    pad_chunk = PKCS7(chunk, block_size)
    # print(text)
    encrypted_bytes += cipher.encrypt(pad_chunk)
  return encrypted_bytes

In [126]:
def PKCS7(input_bytes, block_size):
  pad_length = block_size - len(input_bytes)
  return input_bytes + b'\x04' * pad_length

In [130]:
def encrypt_CBC(bytes_text, key, block_size, initialization_vector):
  block_list = [bytes_text[i * block_size: (i+1) * block_size] for i in range(len(bytes_text) // block_size + 1)]
  
  pre_block = initialization_vector
  result = []
  for block in block_list:
    if len(block) == 0:
      break
    block = PKCS7(block, block_size)
    XOR_block = [b1 ^ b2 for b1, b2 in zip(pre_block, block)]
    #print(f'block: {block}')
    #print(f'XOR_block: {XOR_block} len: {len(XOR_block)}')
    encrypted_block = encrypt_ECB(bytes(XOR_block), key, block_size)
    #print(f'encrypted_block: {[x for x in encrypted_block]}')

    pre_block = encrypted_block
    result += encrypted_block
  return bytes(result)


def decrypt_CBC(bytes_text, key, block_size, initialization_vector):
  block_list = [bytes_text[i * block_size: (i+1) * block_size] for i in range(int(len(bytes_text) / block_size))]
  
  pre_block = initialization_vector
  result = []
  for block in block_list:
    decrypted_block = decrypt_ECB(block, key)
    plain_text = [b1 ^ b2 for b1, b2 in zip(pre_block, decrypted_block)]

    pre_block = block
    result += plain_text
  return bytes(result)

In [ ]:
def encryption_oracle(plain_text: str, block_size: int) -> bytes:
  bytes_text = plain_text.encode()
  random_key = random_aes_key()
  
  append_count = random.randint(5, 10)
  bytes_text = os.urandom(append_count) + bytes_text + os.urandom(append_count)

  encryption_mode = random.randint(0,1)

  if encryption_mode == 0:
    # ECB
    encrypted_bytes = encrypt_ECB(bytes_text, random_key, block_size)
  else:
    # CBC
    encrypted_bytes = encrypt_CBC(bytes_text, random_key, block_size, os.urandom(16))

  return encrypted_bytes, random_key, 'ECB' if encryption_mode == 0 else 'CBC'

def random_aes_key(block_size = 16) -> bytes:
  return os.urandom(block_size)

In [ ]:
def routine(input: str):
  result = dict()
  pairs = input.split('&')
  for pair in pairs:
    key, value = pair.split('=')
    result[key] = value
  return result


def profile_for(input: str):
  input = re.sub(r'\&', '', input)
  input = re.sub(r'\=', '', input)
  params = {'email': input, 'uid': 10, 'role': 'user'}
  return urllib.parse.urlencode(params)

In [ ]:
def new_oracle(plain_text) -> bytes:
  footer_base64 = 'Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkgaGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBqdXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUgYnkK'
  footer_bytes = base64.b64decode(footer_base64)

  if isinstance(plain_text, str):
    bytes_text = random_prefix + plain_text.encode() + footer_bytes
  else:
    bytes_text = random_prefix + plain_text + footer_bytes

  # ECB
  #print("new_oracle", [bytes_text[x*block_size: (x+1)*block_size] for x in range(len(bytes_text))])
  encrypted_bytes = encrypt_ECB(bytes_text, random_key, block_size)

  return encrypted_bytes

In [ ]:
# Step 1
def detect_block_size(plain_text):
  for i in range(10):
    new_text = 'A' * i + plain_text
    ECB_encrypted_text = new_oracle(new_text)
    print(ECB_encrypted_text)
  return block_size

# Step 2
def detect_ECB_mode(plain_text: str, block_size: int):
  target_index = -1
  encrypted_bytes = new_oracle('A'*256 + plain_text)
  max_chunk = len(encrypted_bytes)//block_size

  for i in range(max_chunk - 1):
    for j in range(i + 1, max_chunk):
      if encrypted_bytes[i*block_size: (i+1)*block_size] == encrypted_bytes[j*block_size: (j+1)*block_size]:
        target_index = i
        break
    if target_index != -1:
      break

  return 'ECB' if target_index != -1 else 'UNKNOWN'

# Step 3,4
def craft_input_dict(header: str):
  return {new_oracle(header + bytes([i]))[:block_size]: bytes([i]) for i in range(256)}

# Step 5, 6
def match_output():
  header = b'A' * (block_size - 1)
  result = header

  for x in range(10):
    for i in range(block_size):
      input_dict = craft_input_dict(header)
      key = new_oracle(b'A' * (block_size - 1 - i))[x*block_size:(x+1)*block_size]
      if key in input_dict:
        output = input_dict[key]
      else:
        return result[block_size - 1:]
      result += output
      header = result[-block_size+1:]
  return result[block_size - 1:]

In [ ]:
def get_prefix_length():
  encrypted_btyes_list = []
  for i in range(32):
    encrypted_btyes = new_oracle(b'A' * i)
    print(i, [encrypted_btyes[i*16: (i+1)*16] for i in range(len(encrypted_btyes) // 16 + 1)])
    encrypted_btyes_list.append(encrypted_btyes)

  same_btyes_index = -1
  for j in range(len(encrypted_btyes_list[0])):
    if same_btyes_index != -1:
      break
    for i in range(len(encrypted_btyes_list) - 1):
      #print(encrypted_btyes_list[i][j*block_size: (j+1)*block_size])
      if same_btyes_index == -1:
        if encrypted_btyes_list[i][j*block_size: (j+1)*block_size] == encrypted_btyes_list[i + 1][j*block_size: (j+1)*block_size]:
          if i == 0:
            break
          same_btyes_index = i
  return j, same_btyes_index

def craft_input_dict_14(header: str, prefix_block, prefix_size):
  input_dict = {new_oracle(b'A' * prefix_size + header + bytes([i]))[prefix_block*block_size:(prefix_block+1)*block_size]: bytes([i]) for i in range(256)}
  return input_dict

def match_output_14():
  header = b'A' * (block_size - 1)
  result = header
  prefix_block, prefix_size = get_prefix_length()
  print(prefix_block, prefix_size)

  for x in range(10):
    for i in range(block_size):
      input_dict = craft_input_dict_14(header, prefix_block, prefix_size)
      key = new_oracle(b'A' * prefix_size + b'A' * (block_size - 1 - i))[(x+prefix_block)*block_size:(x+prefix_block+1)*block_size]
      #print(input_dict)
      #print(key)
      if key in input_dict:
        output = input_dict[key]
      else:
        return result[block_size - 1:]
      result += output
      header = result[-block_size+1:]
  return result[block_size - 1:]

In [99]:
def check_and_strip_PKCS7(plain_bytes, block_size):
  if len(plain_bytes) % block_size != 0:
    raise Exception('Wrong size!')

  while True:
    if plain_bytes[-1:] == b'\x04':
      plain_bytes = plain_bytes[:-1]
    elif plain_bytes[-1:] not in string.printable.encode():
      raise Exception('Bad padding!')
    else:
      break
  return plain_bytes

In [160]:
# Step 1
def encrypt_CBC_16(plain_text, key, block_size, initialization_vector):
  prefix = "comment1=cooking%20MCs;userdata="
  suffix = ";comment2=%20like%20a%20pound%20of%20bacon"
  new_text = prefix + plain_text + suffix
  encrypt_bytes = encrypt_CBC(new_text.encode(), key, block_size, initialization_vector)
  return encrypt_bytes

# Step 2
def check_admin_16(encrypt_bytes, key, block_size, initialization_vector):
  decrypt_bytes = decrypt_CBC(encrypt_bytes, key, block_size, initialization_vector)
  print_bytes(decrypt_bytes)
  decrypt_bytes = check_and_strip_PKCS7(decrypt_bytes, block_size)
  return parse_16(decrypt_bytes)

def parse_16(decrypt_bytes):
  s = decrypt_bytes.decode('iso-8859-1')
  print(s)
  block_list = s.split(';')
  for block in block_list:
    key, val = block.split('=')
    if key == 'admin':
      return True
  return False

def print_bytes(text_bytes, block_size = 16):
  print([text_bytes[i:i+block_size] for i in range(0, len(text_bytes), block_size)])

In [162]:
print_bytes(encrypt_bytes)


# print(check_admin_16(encrypt_bytes[:2*block_size]+test[2*block_size:3*block_size]+encrypt_bytes[3*block_size:], random_key, block_size, initialization_vector))

[b'\xfe>|\xcc\x08\x01G\xcb9-$d\x88\xd1\xf1y', b'\x1ea\xbb\xac}#%\xe2\x9c^\x11s?+\xbew', b'\xbc\xe6\x1f\x04\x9a1\xde+\xff\xbc\xca0+&\x9d\x82', b'\xc1\xc1/\xc0\x0c\x9b\xed\x96\x01\x15\xa2\xd4\x18\xda\xc9\xcb', b"8'\xd5Ag4\x7f\xcc\re\x1aG~\xdb\xa2\x1a", b'p\xbf\xd09\n >\x0ff\xa92\x0c\xdf8\x871', b'D\x0b}7\x08V\xe1\xc9\xf6\t\xaf\xe7\xa2k\x9c\\']


# Set_1

Challenge 1

In [1]:
text = '49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d'
ans = 'SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t'

baseInt= int(text, 16)
print(baseInt)

bytes_text = bytes.fromhex(text)
print(bytes_text)

result = base64.b64encode(bytes_text)
print(result)

11259432467145572969189485457381052543241507215288737798329079056359121649591228422793827173000297562297701340508013
b"I'm killing your brain like a poisonous mushroom"
b'SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t'


Challenge 2

In [2]:
buffer1 = '1c0111001f010100061a024b53535009181c'
buffer2 = '686974207468652062756c6c277320657965'
ans = '746865206b696420646f6e277420706c6179'

hex1 = bytes.fromhex(buffer1)
hex2 = bytes.fromhex(buffer2)
print(hex1)
print(hex2)

result = ''
for i in range(len(hex1)):
    result += str(hex(hex1[i] ^ hex2[i]))[2:]

print(result)
print(result == ans)

b'\x1c\x01\x11\x00\x1f\x01\x01\x00\x06\x1a\x02KSSP\t\x18\x1c'
b"hit the bull's eye"
746865206b696420646f6e277420706c6179
True


Challenge 3

In [6]:
text = '1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736'
print(TryDecrypt(text))

(253.57999999999998, "Cooking MC's like a pound of bacon", 88)


Challenge 4

In [7]:
text_list = GetTxt("https://cryptopals.com/static/challenge-data/4.txt")

In [8]:
decrypt_list = []
for text in text_list:
  decrypt_list.append(TryDecrypt(text))
result = sorted(decrypt_list, key=lambda x:x[0], reverse=True)
print(*result[:10], sep="\n")

(219.54000000000002, 'Now that the party is jumping\n', 53)
(113.88999999999997, '[$LrLi;hsDbv!5 j$OaEfG=>rKDl=S', 109)
(113.14999999999999, 'wpUs{gnns F8b-aT2*zRAh{Gj>xuED', 98)
(109.14, "iu`'ern.&nZeM,MjT\x0b:FD~i5+OyEmL", 113)
(105.78999999999999, 'Sw6XSo$^uWyP$C{iUCns3e?E` /]GZ', 108)
(90.35000000000001, 'T2XNi-]1rTIYbP/>2`o%%]T5JonmqA', 99)
(70.12000000000002, ']$4OFs>\\YQ#\nK9y?IT4PeNWTxFqc0\r', 103)
(70.01000000000002, '\\`)]^o}me|~#Am~t[dbF.tq\x0bmAWM|~', 114)
(0, "\x0e6GèY-5QJ\x08\x12CX%6í=æs@Y\x00\x1e?S\\æ'\x102", 0)
(0, '3K\x04\x1dá$÷<\x18\x01\x1aPæ\x08\tzÃ\x08ìîP\x137ì>\x10\x08T \x1d', 0)


Challenge 5

In [9]:
text = '''Burning 'em, if you ain't quick and nimble
I go crazy when I hear a cymbal'''

key = 'ICE'
print(Encrypt(text, key).hex())


0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f


Challenge 6

In [10]:
text_list = GetTxt("https://cryptopals.com/static/challenge-data/6.txt")

In [11]:
text = "".join(text_list)
text = base64.b64decode(text)

print(text)
print(len(text))

b'\x1dB\x1fM\x0b\x0f\x02\x1fO\x13N<\x1aie\x1fI\x1c\x0eN\x13\x01\x0b\x07N\x1b\x01\x16E6\x00\x1e\x01Id T\x1d\x1dC3SNeR\x06\x00GT\x1c\rEM\x07\x04\x0cS\x12<\x0c\x1e\x08I\x1a\t\x11O\x14L!\x1aG+\x00\x05\x1dGY\x11\x04\t\x00d&\x07S\x007\x16\x06\x0c\x1a\x17A\x1d\x01RT0_\x00 \x13\n\x05GO\x12H\x08ENe>\x16\t8E\x06\x05\x08\x1aF\x07O\x1fYx~jb6\x0c\x1d\x0fA\rH\x06U\x1a\x1b\x00\x1dBt\x04\x1e\x01I\x1a\t\x11\x02Rz\x7fI\x00H:\x00\x1a\x13I\x1aOEH\x0f\x1d\rS\x04:\x01R\x19\x01\x0bA\x13\x06\x00L1_Sb\x15\x06\x07\t\x07T\x0b\x17A\x14\x16Iy35\x0b\x1b\x01\x05\x0fF\x07O\x1dNxNH\'R\x04\x07\x0cEXH\x08A\x00O T\x08t\x0b\x1d\x19I\x02\x00\x0e\x16\\\x00R0ie\x1fI\x02\x02T\x00\x01\x0b\x07N\x02\x10S\x01&\x10\x15M\x02\x07\x02\x1fO\x1bNx0i6R\n\x01\tT\x06\x07\tSN\x02\x10S\x08;\x10\x06\x05I\x0f\x0f\x10O;\x00:_G+\x1cId3OT\x02\x10S\x1aO\x05\x16\x11t\x0c\x06M\x0f\x02\x0e\x03CRL=N\x00/\x0bI\r\x08N\x17\r\x15T\x1dO\x0e\x1cE^(\x0bM\x19\x01\x12\x07\nUSxNOb\x06\x01\x0bGS\x1d\x0c\x00\x00\x17\n\x05\x1f\x0c:B^M.\x01A"\x0e\x1cI4VAb5\x06OG*~

In [13]:
test_Normalized = defaultdict(list)
Normalized_list = dict()
for keysize in range(2, 41):
  block_list = []
  for i in range(int(len(text)/keysize) + 1):
    block_list += [text[keysize*i:keysize*(i+1)]]
  
  distance_list = []
  for i in range(len(block_list) - 1):
    distance_list += [HammingDistance(block_list[i],block_list[i+1])]
  Normalized_list[keysize] = sum(distance_list)/len(distance_list)/keysize

  
  for i in range(2, len(distance_list) + 1):
    test_distance_list = distance_list[:i]
    test_Normalized[i] += [(keysize, sum(test_distance_list)/len(test_distance_list)/keysize)]

max_chunk = int(len(text) / 40) + 1
for chunk, normalized_list in test_Normalized.items():
  if chunk > max_chunk:
    break
  print(f"for using {chunk} chunk, best 5 scores are : {sorted(normalized_list, key=lambda x:x[1])[:5]}")

print(*sorted(Normalized_list.items(), key=lambda x:x[1])[:10], sep='\n')
candidate_keysize_list = [x[0] for x in sorted(Normalized_list.items(), key=lambda x:x[1])[:10]]

transpose_block_dict = dict()
for keysize in candidate_keysize_list:
  block_list = []
  for i in range(keysize):
    block_list += [text[i::keysize]]
  transpose_block_dict[keysize] = block_list

print(transpose_block_dict)
for keysize, transpose_block_list in transpose_block_dict.items():
  decrypt_list = []
  for index, transpose_block in enumerate(transpose_block_list):
    score, _, key = TryDecrypt(transpose_block)
    decrypt_list.append((index, (score, key)))
  #print(decrypt_list)

  key = "".join([chr(x[1][1]) for x in decrypt_list])
  if all([x in string.printable for x in key]):
    print(f"key: {key}")

for using 2 chunk, best 5 scores are : [(2, 2.25), (5, 2.3), (3, 2.3333333333333335), (18, 2.8333333333333335), (20, 2.85)]
for using 3 chunk, best 5 scores are : [(2, 2.0), (3, 2.6666666666666665), (29, 2.793103448275862), (5, 2.8), (18, 2.9444444444444446)]
for using 4 chunk, best 5 scores are : [(2, 2.25), (5, 2.6), (29, 2.8189655172413794), (3, 2.8333333333333335), (18, 3.0555555555555554)]
for using 5 chunk, best 5 scores are : [(2, 2.4), (5, 2.52), (3, 2.8000000000000003), (29, 2.8344827586206898), (31, 3.0258064516129033)]
for using 6 chunk, best 5 scores are : [(2, 2.5833333333333335), (5, 2.7), (3, 2.8333333333333335), (29, 2.8563218390804597), (18, 3.0185185185185186)]
for using 7 chunk, best 5 scores are : [(5, 2.7142857142857144), (29, 2.8226600985221677), (3, 2.857142857142857), (15, 2.99047619047619), (18, 3.0238095238095237)]
for using 8 chunk, best 5 scores are : [(5, 2.775), (29, 2.8189655172413794), (3, 2.8333333333333335), (2, 3.0), (15, 3.0833333333333335)]
for usin

Chellange 7

In [14]:
text_list = GetTxt("https://cryptopals.com/static/challenge-data/7.txt")
text = ''.join(text_list)
text = base64.b64decode(text)
key = b'YELLOW SUBMARINE'

In [17]:
print(decrypt_ECB(text, key))

b"I'm back and I'm ringin' the bell \nA rockin' on the mike while the fly girls yell \nIn ecstasy in the back of me \nWell that's my DJ Deshay cuttin' all them Z's \nHittin' hard and the girlies goin' crazy \nVanilla's on the mike, man I'm not lazy. \n\nI'm lettin' my drug kick in \nIt controls my mouth and I begin \nTo just let it flow, let my concepts go \nMy posse's to the side yellin', Go Vanilla Go! \n\nSmooth 'cause that's the way I will be \nAnd if you don't give a damn, then \nWhy you starin' at me \nSo get off 'cause I control the stage \nThere's no dissin' allowed \nI'm in my own phase \nThe girlies sa y they love me and that is ok \nAnd I can dance better than any kid n' play \n\nStage 2 -- Yea the one ya' wanna listen to \nIt's off my head so let the beat play through \nSo I can funk it up and make it sound good \n1-2-3 Yo -- Knock on some wood \nFor good luck, I like my rhymes atrocious \nSupercalafragilisticexpialidocious \nI'm an effect and that you can bet \nI can take 

Challenge 8

In [18]:
text_list = GetTxt("https://cryptopals.com/static/challenge-data/8.txt")

In [19]:
print(len(text_list))

205


In [20]:
target_index = 0
for index, text in enumerate(text_list):
  text = bytes.fromhex(text)
  if len(text) < 1:
    continue
  for i in range(10):
    for j in range(i + 1, 10):
      if text[i*16: (i+1)*16] == text[j*16: (j+1)*16]:
        print(index, text)
        target_index = index
        break

print(text_list[target_index])

132 b'\xd8\x80a\x97@\xa8\xa1\x9bx@\xa8\xa3\x1c\x81\n=\x08d\x9a\xf7\r\xc0oO\xd5\xd2\xd6\x9ctL\xd2\x83\xe2\xdd\x05/kd\x1d\xbf\x9d\x11\xb04\x85B\xbbW\x08d\x9a\xf7\r\xc0oO\xd5\xd2\xd6\x9ctL\xd2\x83\x94u\xc9\xdf\xdb\xc1\xd4e\x97\x94\x9d\x9c~\x82\xbfZ\x08d\x9a\xf7\r\xc0oO\xd5\xd2\xd6\x9ctL\xd2\x83\x97\xa9>\xab\x8dj\xec\xd5fH\x91Tx\x9ak\x03\x08d\x9a\xf7\r\xc0oO\xd5\xd2\xd6\x9ctL\xd2\x83\xd4\x03\x18\x0c\x98\xc8\xf6\xdb\x1f*?\x9c@@\xde\xb0\xabQ\xb2\x993\xf2\xc1#\xc5\x83\x86\xb0o\xba\x18j'
132 b'\xd8\x80a\x97@\xa8\xa1\x9bx@\xa8\xa3\x1c\x81\n=\x08d\x9a\xf7\r\xc0oO\xd5\xd2\xd6\x9ctL\xd2\x83\xe2\xdd\x05/kd\x1d\xbf\x9d\x11\xb04\x85B\xbbW\x08d\x9a\xf7\r\xc0oO\xd5\xd2\xd6\x9ctL\xd2\x83\x94u\xc9\xdf\xdb\xc1\xd4e\x97\x94\x9d\x9c~\x82\xbfZ\x08d\x9a\xf7\r\xc0oO\xd5\xd2\xd6\x9ctL\xd2\x83\x97\xa9>\xab\x8dj\xec\xd5fH\x91Tx\x9ak\x03\x08d\x9a\xf7\r\xc0oO\xd5\xd2\xd6\x9ctL\xd2\x83\xd4\x03\x18\x0c\x98\xc8\xf6\xdb\x1f*?\x9c@@\xde\xb0\xabQ\xb2\x993\xf2\xc1#\xc5\x83\x86\xb0o\xba\x18j'
132 b'\xd8\x80a\x97@\xa8\xa1\x

# Set_2

Challenge 9

In [22]:
PKCS7("YELLOW SUBMARINE", 20)

b'YELLOW SUBMARINE\x04\x04\x04\x04'

Challenge 10

In [24]:
plain_text_list = GetTxt("https://cryptopals.com/static/challenge-data/10.txt")
plain_text = ''.join(text_list)
key = b'YELLOW SUBMARINE'
print(plain_text)

...


In [25]:
block_size = 16
initialization_vector = b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

print(decrypt_CBC(plain_text, key, block_size, initialization_vector).decode('utf-8'))

TypeError: ignored

Challenge 11

In [27]:
block_size = 16
for i in range(10):
  encrypted_bytes, key, encryption_mode = encryption_oracle('A'*256, block_size)

  target_index = -1
  max_chunk = int(len(encrypted_bytes)/block_size)
    
  for i in range(max_chunk - 1):
    for j in range(i + 1, max_chunk):
      if encrypted_bytes[i*block_size: (i+1)*block_size] == encrypted_bytes[j*block_size: (j+1)*block_size]:
        #print(encrypted_bytes)
        #print(encrypted_bytes[i*block_size: (i+1)*block_size], encrypted_bytes[j*block_size: (j+1)*block_size], i, j)
        target_index = i
        break
    if target_index != -1:
      break
  
  detected_mode = 'ECB' if target_index != -1 else 'CBC'
  print(f'{encryption_mode == detected_mode}, answer: {encryption_mode}, detected: {detected_mode}')

True, answer: CBC, detected: CBC
True, answer: CBC, detected: CBC
True, answer: CBC, detected: CBC
True, answer: CBC, detected: CBC
True, answer: ECB, detected: ECB
True, answer: ECB, detected: ECB
True, answer: ECB, detected: ECB
True, answer: ECB, detected: ECB
True, answer: ECB, detected: ECB
True, answer: ECB, detected: ECB


Challenge 12

In [30]:
block_size = 16
random_key = os.urandom(block_size)
random_bytes = os.urandom(random.randint(5, 10))

encrypted_bytes = new_oracle(plain_text)
print(encrypted_bytes)

# Step 1 Get block_size
#detected_block_size = detect_block_size(plain_text)
#print(f'block size detection:{detected_block_size==block_size} block_size:{block_size} detected_block_size:{detected_block_size}')
# Step 2 Detected ECB mode
detected_mode = detect_ECB_mode(plain_text, block_size)
print(f'detected_mod: {detected_mode}')
# Step 3,4 Craft input
# Step 5,6 match_output
match_output()

NameError: ignored

Challenge 13

In [32]:
block_size = 16
profile = profile_for('foo@bar.com')
print(f'profile: {profile}')
key = random_aes_key()
print(key)
encrypt_profile = encrypt_ECB(str.encode(profile), key, block_size)
print(encrypt_profile)
decrypt_profile = decrypt_ECB(encrypt_profile, key)
print(decrypt_profile)

target = 'email=foo%40bar.com&uid=10&role=admin'
print(f'target: {target}')
encrypt_target = encrypt_ECB(str.encode(target), key, block_size)
print(encrypt_target)
decrypt_target = decrypt_ECB(encrypt_target, key)
print(decrypt_target)

profile: email=foo%40bar.com&uid=10&role=user
b'\xd4V\x962\x18\xe1\xe7\xe6+\x03\xe9_\xdcp\x90\x90'
b'"4\x80\x1fJgH\xda\x92\xd6}:=PO\xae\xc6\xca\xf8|\x98\r\xbcr\x1d\'T\xe3\xf6\x96\xbbD\xec\xe2r\xed\x86\xc1D\xbd\xc7\x94\xca4m\x10\xea\xba'
b'email=foo%40bar.com&uid=10&role=user\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
target: email=foo%40bar.com&uid=10&role=admin
b'"4\x80\x1fJgH\xda\x92\xd6}:=PO\xae\xc6\xca\xf8|\x98\r\xbcr\x1d\'T\xe3\xf6\x96\xbbD\x88\x01=\xea\x05a\xe3Y%\xb8\x0c]\xbc\x12\xd5c'
b'email=foo%40bar.com&uid=10&role=admin\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'


In [33]:
origin = profile_for('foo@bar.com')
print([origin[i*16:(i+1)*16] for i in range(5)])
encrypt_origin = encrypt_ECB(str.encode(origin), key, block_size)
print([encrypt_origin[i*16:(i+1)*16] for i in range(5)])
hack = profile_for('foo@bar.admin')
print([hack[i*16:(i+1)*16] for i in range(5)])
encrypt_hack = encrypt_ECB(str.encode(hack), key, block_size)
print([encrypt_hack[i*16:(i+1)*16] for i in range(5)])

['email=foo%40bar.', 'com&uid=10&role=', 'user', '', '']
[b'"4\x80\x1fJgH\xda\x92\xd6}:=PO\xae', b"\xc6\xca\xf8|\x98\r\xbcr\x1d'T\xe3\xf6\x96\xbbD", b'\xec\xe2r\xed\x86\xc1D\xbd\xc7\x94\xca4m\x10\xea\xba', b'', b'']
['email=foo%40bar.', 'admin&uid=10&rol', 'e=user', '', '']
[b'"4\x80\x1fJgH\xda\x92\xd6}:=PO\xae', b'\x84L\xeaB\xd7\xe3\xf9%W\xab\xb4\x8d\x80\x84\xd6\xf5', b'f[\x0c\n\xbaG\xd7rR\xe0\xbae\xde\x8e\xa1(', b'', b'']


In [34]:
encrypt_origin = encrypt_origin.replace(encrypt_origin[32:48], encrypt_hack[16:32]) + encrypt_hack[32:48]
print(encrypt_origin)
print(decrypt_ECB(encrypt_origin, key))

b'"4\x80\x1fJgH\xda\x92\xd6}:=PO\xae\xc6\xca\xf8|\x98\r\xbcr\x1d\'T\xe3\xf6\x96\xbbD\x84L\xeaB\xd7\xe3\xf9%W\xab\xb4\x8d\x80\x84\xd6\xf5f[\x0c\n\xbaG\xd7rR\xe0\xbae\xde\x8e\xa1('
b'email=foo%40bar.com&uid=10&role=admin&uid=10&role=user\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'


In [35]:
hack = profile_for('foo@bar.admin')
print([hack[i*16:(i+1)*16] for i in range(5)])
encrypt_hack = encrypt_ECB(str.encode(hack), key, block_size)
print([encrypt_hack[i*16:(i+1)*16] for i in range(5)])

['email=foo%40bar.', 'admin&uid=10&rol', 'e=user', '', '']
[b'"4\x80\x1fJgH\xda\x92\xd6}:=PO\xae', b'\x84L\xeaB\xd7\xe3\xf9%W\xab\xb4\x8d\x80\x84\xd6\xf5', b'f[\x0c\n\xbaG\xd7rR\xe0\xbae\xde\x8e\xa1(', b'', b'']


Challenge 14

In [36]:
random_prefix = os.urandom(random.randint(1, 32))
print(random_prefix, len(random_prefix))
print(random_key)

b'\x81\xf2\xfd\xd1G\x1c\x1f$\xb9\x991\x83P' 13
b'\x82\xe9Wv\xf1\x93\x04P\xd1^\xd5\x0e\x17C\xd4\x15'


In [37]:
attacker_controlled = b'attacker_controlled'
encrypted_btyes = new_oracle(random_prefix + attacker_controlled)
print(encrypted_btyes)

b"\x00\xb5\xa4\xe6\xa4\x0b\xefN\xbc'\x1c=N6\x83\xcc p\xc6z\xba\x14\twD\x00\x12\x9dD\xfd\x99\xd3\xaf\r\x1f\x1c|\x91\x02X\xb4fm\xa3\xf3\x10\xe9\x83_p\xa6\xcb\x8c\xb9\xe3\xb6c\x8e\xee\x11\xb2\x90\x19\xa4\x8c\xb0\x8fK\xae\xbf\xd8\xf1U\xe9p\x8b\xd8\xc5z\xd6%e\x06\r\xe9\xd3\xc1\x9d\xa3\x06\x80]\xfc\x18\x9d!\x90\x1aj\x05v\x0cW\x1e\x07\x9c*\xad\xc2\x95\xdcc\xa1\x98@*\x19\xb8\xd2\xe8\xf3\x0e\x89#X\x93\xcbhw\xe1\xed\x1f\xce[K\x905\xfc\xeeECK\xddx^\xc8\xce\xc4\xad\x9d\x9fg=\x1dC\xf5\x9f\xf4\x156\x11r-=\x1a\xa7C\xee%\xb8\x14\x1bU#E\x07\xf4\xe4if^66\x7f\xab\xdf\xbd\x14\xb0\x9f\xa2\x16"


In [39]:
match_output_14()

0 [b'\xbb>)\xc4\xeee\x05\x12W\x0cT\xde\x05\xe0\xb9\x9a', b'_p\xa6\xcb\x8c\xb9\xe3\xb6c\x8e\xee\x11\xb2\x90\x19\xa4', b'\x8c\xb0\x8fK\xae\xbf\xd8\xf1U\xe9p\x8b\xd8\xc5z\xd6', b'%e\x06\r\xe9\xd3\xc1\x9d\xa3\x06\x80]\xfc\x18\x9d!', b'\x90\x1aj\x05v\x0cW\x1e\x07\x9c*\xad\xc2\x95\xdcc', b'\xa1\x98@*\x19\xb8\xd2\xe8\xf3\x0e\x89#X\x93\xcbh', b'w\xe1\xed\x1f\xce[K\x905\xfc\xeeECK\xddx', b'^\xc8\xce\xc4\xad\x9d\x9fg=\x1dC\xf5\x9f\xf4\x156', b'\x11r-=\x1a\xa7C\xee%\xb8\x14\x1bU#E\x07', b'\xf4\xe4if^66\x7f\xab\xdf\xbd\x14\xb0\x9f\xa2\x16', b'']
1 [b'vGx\xe6\xe7W^b\xe7Oq\xc6R\xfd\xf5\xef', b'\x00?,\x89o.\xa8\x0fx\x18\xf6\xa6\xf0\xc9\x07\xd8', b'6Xu\xa4\xdaj\xc9;\x99U\xd8\xcacEWI', b'\xad\xa9\xbf`l\x06\x86\x9b\xc4\x1fRI}\x08\xbe,', b'<\x84\xe4\x1d\x9a\x1c\xd8i\xa7}<A\x93\r\xd9V', b"\xc7'\x92\xc8\xfd\x15_\x064s\xfd\xb1m.\x0c\x99", b'\x0b\xfcZ\x1d\xf7\xb1^\x93BaU<`\xbc<\xd3', b'\xf1T\xf0\xbc\xfa\xfb\x7f\x96\xf4\xfa\xbb&\xf1\x9c\xe94', b'\x1e\xe0\xa1`\xcc\xff5\x80\n\xe8X/3$M\x98', b'9v\xde\xf4\xa5\x9e

b"Rollin' in my 5.0\nWith my rag-top down so my hair can blow\nThe girlies on standby waving just to say hi\nDid you stop? No, I just drove by\n\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00"

Challenge 15

In [60]:
plain_bytes = b"ICE ICE BABY\x04\x04\x04\x04"
print(check_and_strip_PKCS7(plain_bytes, block_size))
plain_bytes = b"ICE ICE BABY\x05\x05\x05\x05"
# check_and_strip_PKCS7(plain_bytes, block_size)

b'ICE ICE BABY'


Challenge 16

In [167]:
block_size = 16
ramdom_key = random_aes_key()
initialization_vector = random_aes_key()
plain_text = "ABCDEFGHIJKLMNOP" * 2

encrypt_bytes = encrypt_CBC_16(plain_text, random_key, block_size, initialization_vector)
print_bytes(encrypt_bytes)
print(check_admin_16(encrypt_bytes, random_key, block_size, initialization_vector))

target_chunk = encrypt_bytes[2*block_size:3*block_size]
target_chunk = [b1 ^ b2 for b1, b2 in zip(target_chunk, b';admin=true;AAAA')]
target_chunk = bytes([b1 ^ b2 for b1, b2 in zip(target_chunk, b"ABCDEFGHIJKLMNOP")])

print(check_admin_16(encrypt_bytes[:2*block_size] + target_chunk + encrypt_bytes[3*block_size:], random_key, block_size, initialization_vector))

[b'\x0f\\\x0bNjlU\xd5\x0c\xfc;\xdd,\xc8\xa8:', b'\x03@\xaa\x89Lyv\x94\xa4[\xfe\xb8\xeby\xd6\xf3', b'\xe5!5\x1fW\x95\xdc#\xcf\x89\x8b\xc7<\xce\xb8j', b'8~\xe9\xd9\x18\xda\xcd\xbf\xff\xd5\xff\xeb\x04s\xde\x97', b'\xc8\x1f\x84\xf7\xd9\x00\xf6\x9fz\xd2\x98@\x81\x16d[', b'\xf0"\xf3\x93\xa2\xfcP\xacM\xcc\x1cS\xec}\xaa\xbd', b'TN\xc4\x17P)\xd6\xd3\xf4\x8a87\xfa,\xeb}']
[b'comment1=cooking', b'%20MCs;userdata=', b'ABCDEFGHIJKLMNOP', b'ABCDEFGHIJKLMNOP', b';comment2=%20lik', b'e%20a%20pound%20', b'of%20bacon\x04\x04\x04\x04\x04\x04']
comment1=cooking%20MCs;userdata=ABCDEFGHIJKLMNOPABCDEFGHIJKLMNOP;comment2=%20like%20a%20pound%20of%20bacon
False
[b'comment1=cooking', b'%20MCs;userdata=', b'\xa2\xca\x01\xfdX!\x1cN\xe7\xc8hh\xa1\xe2\xc3\xb7', b';admin=true;AAAA', b';comment2=%20lik', b'e%20a%20pound%20', b'of%20bacon\x04\x04\x04\x04\x04\x04']
comment1=cooking%20MCs;userdata=¢ÊýX!NçÈhh¡âÃ·;admin=true;AAAA;comment2=%20like%20a%20pound%20of%20bacon
True
